In [1]:
import requests, json, time, math
from datetime import datetime
from multiprocessing import Pool
from sqlalchemy import *
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.auto import tqdm

#### Good to have

In [2]:
def split_list(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [3]:
for i in tqdm(range(10)):
    time.sleep(.3)

#### Get user inventory
##### Option A: For Loop

In [4]:
path_user_id = 'data/steam_user_id.txt'
with open(path_user_id, 'rb') as f:
    lst_user_id = f.readlines()

lst_user_id[:5]

FileNotFoundError: [Errno 2] No such file or directory: 'data/steam_user_id.txt'

In [ ]:
'\n76561198158086086 \n\n\t'.strip()

In [ ]:
for user_id in lst_user_id[:5]:
    base_url = 'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/'
    params = {
        'key' : 'D0C62157A8941F12A687382B6D635449',
        'steamid' : user_id.strip(),
        'format' : 'json' 
    }
    r = requests.get(base_url, params = params, headers = {})
    user_inventory = r.json().get('response').get('games')
    time.sleep(.5)
    print(user_id, '\n', user_inventory)

##### Option B: Multiprocessing

In [ ]:
# what is multiprocessing?
# Multiprocessing vs threading, queue

In [ ]:
path_user_id = 'data/steam_user_id.txt'
with open(path_user_id, 'r') as f:
    lst_user_id = f.readlines()[:50]

In [ ]:
def worker(lst_user_id_temp):
    dic_temp = {}
    for user_id in tqdm(lst_user_id_temp, leave=False):
        base_url = 'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/'
        params = {
            'key' : 'D0C62157A8941F12A687382B6D635449',
            'steamid' : user_id.strip(),
            'format' : 'json' }
        r = requests.get(base_url, params = params)
        user_inventory = r.json().get('response').get('games')
        dic_temp.update({user_id.strip():user_inventory})
        time.sleep(.5)
    return dic_temp

In [ ]:
p = Pool(2)

dic_master = {}
for i in tqdm(list(split_list(lst_user_id,10))):
    lst_temp_dic = p.map(worker, split_list(i,5))
    for j in lst_temp_dic:
        dic_master.update(j)
    
    time.sleep(5)

In [ ]:
with open('data/crawled_user_inventory.txt', 'w') as f:
    for user_id, user_inventory in list(dic_master.items()):
        f.write(json.dumps({str(user_id):user_inventory}))
        f.write('\n')

### Web Crawler II
#### 1) rate limit
#### 2) Headers, cookies
#### 3) Multiprocessing / Threading
#### 4) Selenium

In [ ]:
r = requests.get('https://www.youtube.com/watch?v=sDF1fHQO42M')
soup = BeautifulSoup(r.text, 'lxml')
soup.find_all('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'})

In [ ]:
from selenium import webdriver

In [ ]:
driver = webdriver.Chrome('/Applications/chromedriver')

In [ ]:
driver.get('https://www.youtube.com/watch?v=h31myLyc_qk')

In [ ]:
soup = BeautifulSoup(driver.page_source, 'lxml')
soup.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).string

#### get app info

In [ ]:
# get all available app id
url = 'https://api.steampowered.com/ISteamApps/GetAppList/v2/'
r = requests.get(url)
dic_app_list = r.json()
lst_app_id = [i.get('appid') for i in dic_app_list.get('applist').get('apps')]
len(lst_app_id)

In [ ]:
current_count = 0
path_app_detail_sample = 'app_detail_sample.txt'
with open(path_app_detail_sample, 'w') as f:
    for app_id in tqdm(lst_app_id[:5]):
        url_app_detail = ('http://store.steampowered.com/api/appdetails?appids=%s') % (app_id)
        for i in range(3):
            try:
                r = requests.get(url_app_detail)
                result = r.json()
                break
            except Exception as e:
                print(e)
                time.sleep(5)
        f.write(json.dumps(result))
        f.write('\n')

        if current_count > 0 and current_count % 200 == 0:
            time.sleep(300)
        else:
            time.sleep(.5)

In [ ]:
path_app_detail = 'data/app_detail.txt'

with open(path_app_detail, 'r') as f:
    
    dic_steam_app = {
        'initial_price':{},
        'name':{},
        'score':{},
        'windows':{},
        'mac':{},
        'linux':{},
        'type':{},
        'release_date':{},
        'recommendation':{},
        'header_image':{}
    }
    lst_raw_string = f.readlines()[:200]

    for raw_string in tqdm(lst_raw_string):
        try:
            app_data = list(json.loads(raw_string).values())[0]
            if app_data.get('success'):
                app_data = app_data.get('data')
                steam_id = app_data.get('steam_appid')
                initial_price = app_data.get('price_overview',{}).get('initial')
                if app_data.get('is_free') == True:
                    initial_price = 0
                app_name = app_data.get('name')
                critic_score = app_data.get('metacritic', {}).get('score')
                app_type = app_data.get('type')
                for (platform, is_supported) in app_data.get('platforms',{}).items():
                    if is_supported == True:
                        dic_steam_app[platform].update({steam_id:1})
                    else:
                        dic_steam_app[platform].update({steam_id:0})
                if app_data.get('release_date',{}).get('coming_soon') == False:
                    release_date = app_data.get('release_date',{}).get('date')
                    if not release_date == '':
                        try:
                            release_date = datetime.strptime(release_date, '%b %d, %Y')
                        except Exception as e:
                            try:
                                release_date = datetime.strptime(release_date, '%d %b, %Y')
                            except:
                                release_date = None
                        

                recommendation = app_data.get('recommendations',{}).get('total')
                header_image = app_data.get('header_image')
                dic_steam_app['initial_price'].update({steam_id:initial_price})
                dic_steam_app['name'].update({steam_id:app_name})
                dic_steam_app['score'].update({steam_id:critic_score})
                dic_steam_app['type'].update({steam_id:app_type})
                dic_steam_app['release_date'].update({steam_id:release_date})
                dic_steam_app['recommendation'].update({steam_id:recommendation})
                dic_steam_app['header_image'].update({steam_id:header_image})
            time.sleep(.1)
        except:
            pass


#### Work with MySQL in Python

In [ ]:
df_app_info = pd.DataFrame(dic_steam_app)
df_app_info.index.name = 'app_id'
df_app_info.reset_index(inplace=True)
df_app_info.head()

In [ ]:
user = ''
password = ''
host = '127.0.0.1'
db_name = 'steam'
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{db_name}?charset=utf8mb4')

In [ ]:
df_app_info.to_sql('tbl_app_info', engine, if_exists='replace',index=False)

In [ ]:
engine.execute(
    '''
    select * from tbl_app_info limit 10
    ''').fetchall()

In [ ]:
pd.read_sql_query('''
    select * from tbl_app_info limit 10
''', engine)